In [1]:
import json
import os
from tqdm import tqdm
from glob import glob
from multiprocess import Pool
import itertools
import string

printable = set(string.printable)
timestamps = [i * 0.02 for i in range(1500 + 1)]

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

In [2]:
# folders = [
#     'klasik_processed',
#     'malaysian-podcast_processed',
#     'sg-podcast_processed',
#     'filtered-24k_processed',
#     'parlimen-24k-chunk_processed',
# ]

# audio = []
# for f in folders:
#     audio.extend(glob(os.path.join(f, '**/*.mp3'), recursive = True))
    
    
# def loop(files):
#     files, _ = files
#     for f in tqdm(files):
#         new_f = ''.join([c for c in f if c in printable])
#         if new_f != f:
#             folder = os.path.split(new_f)[0]
#             os.makedirs(folder, exist_ok = True)
#             os.replace(f, new_f)
            
            
# multiprocessing(audio, loop, cores = 20, returned = False)

In [3]:
files = glob('*_alignment/**/*.alignment', recursive = True)
files = [f for f in files if 'prepared-pseudolabel' not in f]
len(files)

2449331

In [4]:
files[0]

'klasik_processed_alignment/RUMAH NORDIN AHMAD [iATLCGdiHVw]/RUMAH NORDIN AHMAD [iATLCGdiHVw]_22.alignment'

In [12]:
from collections import Counter

def detect_repeated_phrases(text, n=3):
    words = text.lower().split()
    phrases = [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]
    counter = Counter(phrases)
    return [phrase for phrase, count in counter.items() if count > 2]

def loop(files):
    files, _ = files
    filtered = []
    for f in tqdm(files):
        with open(f) as fopen:
            d = json.load(fopen)
        
        if d[-1]['start'] <= 4:
            continue
        
        scores = [d_['score'] for d_ in d if d_['score'] <= -12]
        if len(scores):
            continue
        
        t_ = ' '.join([d_['text'] for d_ in d])
        if len(detect_repeated_phrases(t_, n=3)):
            continue
        
        filtered.append(f)
        
    return filtered

In [13]:
filtered = multiprocessing(files, loop, cores = 30)

100%|██████████████████████████████████████████████████████████████████████████████| 81644/81644 [00:02<00:00, 29972.68it/s]


In [14]:
len(filtered)

599922

In [15]:
from collections import defaultdict

counts = defaultdict(int)
selected = defaultdict(list)
for f in tqdm(filtered):
    new_f = ''.join([c for c in f if c in printable])
    audio_filename = new_f.replace('_alignment/', '/').replace('.alignment', '.mp3')
    if not os.path.exists(audio_filename):
        continue
    
    counts[f.split('/')[0]] += 1
    selected[f.split('/')[0]].append({
        'alignment_filename': f,
        'audio_filename': audio_filename,
    })
counts

100%|███████████████████████████████████████████████████████████████████████████| 599922/599922 [00:05<00:00, 100355.08it/s]


defaultdict(int,
            {'klasik_processed_alignment': 4125,
             'malaysian-podcast_processed_alignment': 27982,
             'sg-podcast_processed_alignment': 6708,
             'filtered-24k_processed_alignment': 217029,
             'parlimen-24k-chunk_processed_alignment': 268365})

In [16]:
import random

data = []
for k, v in selected.items():
    if len(v) > 200000:
        rows = random.sample(v, 200000)
    else:
        rows = v
    for row in tqdm(rows):
        data.append({**row})

100%|██████████████████████████████████████████████████████████████████████████| 200000/200000 [00:00<00:00, 1220761.33it/s]


In [17]:
len(data)

438815

In [18]:
data[0]['audio_filename']

{'alignment_filename': 'klasik_processed_alignment/RUMAH NORDIN AHMAD [iATLCGdiHVw]/RUMAH NORDIN AHMAD [iATLCGdiHVw]_22.alignment',
 'audio_filename': 'klasik_processed/RUMAH NORDIN AHMAD [iATLCGdiHVw]/RUMAH NORDIN AHMAD [iATLCGdiHVw]_22.mp3'}

In [25]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-bahasa-en", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)

fasttext.ftz:   0%|          | 0.00/331M [00:00<?, ?B/s]

In [66]:
import re
from xml.etree.ElementTree import Element, SubElement, tostring
import xml.dom.minidom

pattern = re.compile(r'<\|([\d.]+)\|>([^<]*)')

def format_srt_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{secs:02},{millis:03}"

def get_subtitles(input_text):
    entries = pattern.findall(input_text)

    subtitles = []
    for i in range(len(entries) - 1):
        start_time = float(entries[i][0])
        end_time = float(entries[i + 1][0])
        text = entries[i][1].strip()
        if text:
            subtitles.append((start_time, end_time, text))
            
    return subtitles

def get_srt(subtitles):
    srt_output = ""
    for idx, (start, end, text) in enumerate(subtitles, start=1):
        srt_output += f"{idx}\n"
        srt_output += f"{format_srt_time(start)} --> {format_srt_time(end)}\n"
        srt_output += f"{text}\n\n"
    return srt_output

def format_ttml_time(seconds):
    return f"{int(seconds // 3600):02}:{int((seconds % 3600) // 60):02}:{int(seconds % 60):02}.{int((seconds % 1) * 1000):03}"

def get_tt(subtitles):
    tt = Element('tt', xmlns="http://www.w3.org/ns/ttml")
    body = SubElement(tt, 'body')
    div = SubElement(body, 'div')

    for start, end, text in subtitles:
        SubElement(div, 'p', begin=format_ttml_time(start), end=format_ttml_time(end)).text = text

    xml_str = xml.dom.minidom.parseString(tostring(tt)).toprettyxml(indent="  ")
    return str(xml_str)

word = [
    'audio to Whisper ASR format word timestamp',
    'transcribe the audio into Whisper format in word timestamp'
]

srt = [
    'audio to SRT format',
    'transcribe the audio into srt format',
]

ttml = [
    'audio to TTML format',
    'transcribe the audio into ttml format',
]

segment = [
    'audio to Whisper ASR format',
    'transcribe the audio into Whisper format'
]

transcribe = ['transcribe the audio']

def loop(rows):
    rows, _ = rows
    data = []
    for row in tqdm(rows):
        with open(row['alignment_filename']) as fopen:
            c = json.load(fopen)
        min_t = min([c_['start'] for c_ in c])
        segments, temp = [], [c[0]]
        last_t = c[0]['end']
        for c_ in c[1:]:
            if (c_['start'] - last_t) > 0.3:
                segments.append(temp)
                temp = []

            last_t = c_['end']
            temp.append(c_)

        if len(temp):
            segments.append(temp)
        
        ts = []
        for s in segments:
            start = min(timestamps, key=lambda t: abs(t - (s[0]['start'] - min_t)))
            end = min(timestamps, key=lambda t: abs(t - (s[-1]['end'] - min_t)))
            w = ' '.join([c_['text'] for c_ in s])
            t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
            ts.append(t)
        
        ts = ''.join(ts)
        
        cleaned_text = re.sub(r"<\|.*?\|>", "", ts).strip()
        if lang_model.predict(cleaned_text)[0][0] == '__label__english':
            predict_lang = 'en'
        else:
            predict_lang = 'ms'
        
        new_text = f"<|startoftranscript|><|{predict_lang}|><|transcribe|>{ts}<|endoftext|>"
            
        data.append({
            'question': random.choice(segment),
            'answer': new_text,
            'audio_filename': row['audio_filename'],
        })
        
        try:
            input_text = new_text.split('<|transcribe|>')[1]
            subtitles = get_subtitles(input_text)
            
            if random.random() > 0.9:
                try:
                    data.append({
                        'question': random.choice(srt),
                        'answer': get_srt(subtitles),
                        'audio_filename': row['audio_filename'],
                    })
                except:
                    pass
            
            if random.random() > 0.9:
                try:
                    data.append({
                        'question': random.choice(ttml),
                        'answer': get_tt(subtitles),
                        'audio_filename': row['audio_filename'],
                    })
                except Exception as e:
                    print(e)
                    pass
        except:
            pass
        
        if random.random() > 0.4:
            ts = []

            for c_ in c:
                start = min(timestamps, key=lambda t: abs(t - (c_['start'] - min_t)))
                end = min(timestamps, key=lambda t: abs(t - (c_['end'] - min_t)))
                w = c_['text']
                t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
                ts.append(t)

            ts = ''.join(ts)
            ts = f"<|startoftranscript|><|{predict_lang}|><|transcribeprecise|>{ts}<|endoftext|>"
            
            data.append({
                'question': random.choice(word),
                'answer': ts,
                'audio_filename': row['audio_filename'],
            })
            
    return data

In [67]:
alignment = loop((data[:10], 0))

100%|██████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 313.87it/s]


In [68]:
alignment

[{'question': 'transcribe the audio into Whisper format',
  'answer': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Sangat menarik sebenarnya rumah ini kerana<|1.58|><|2.40|> di dalam rumah ini sebenarnya ini adalah<|4.44|><|4.78|> replika ataupun bukan rumah sebenar.<|7.02|><|endoftext|>',
  'audio_filename': 'klasik_processed/RUMAH NORDIN AHMAD [iATLCGdiHVw]/RUMAH NORDIN AHMAD [iATLCGdiHVw]_22.mp3'},
 {'question': 'audio to Whisper ASR format',
  'answer': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Walaupun rumah ini dijadikan rumah inap desa<|2.44|><|3.08|> dan dalam usaha mendekatkan peminat dengan Nordin Ahmad,<|6.18|><|6.56|> pengunjung<|7.14|><|7.56|> dapat bermalam atau tinggal di sini<|9.44|><|9.88|> dengan kemudahan yang serba lengkap.<|11.80|><|endoftext|>',
  'audio_filename': 'klasik_processed/RUMAH NORDIN AHMAD [iATLCGdiHVw]/RUMAH NORDIN AHMAD [iATLCGdiHVw]_13.mp3'},
 {'question': 'audio to SRT format',
  'answer': '1\n00:00:00,000 --> 00:00:02,439\nWalaupun 

In [69]:
alignment = multiprocessing(data, loop, cores = 50)

100%|██████████████████████████████████████████████████████████████████████████████████| 8776/8776 [00:38<00:00, 229.94it/s]


In [70]:
len(alignment)

790737

In [71]:
alignment[-10:]

[{'question': 'audio to Whisper ASR format',
  'answer': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Dengan mengambil kira dan memperhalusi perkara-perkara di atas,<|3.16|><|3.60|> pihak Kementerian<|4.44|><|5.00|> akan seterusnya meneliti<|6.68|><|endoftext|>',
  'audio_filename': 'parlimen-24k-chunk_processed/parlimen-24k-LANGSUNG  Persidangan Dewan Rakyat 22 November 2023  Sesi Pagi [S1iU7QQQ-BU]_000/parlimen-24k-LANGSUNG  Persidangan Dewan Rakyat 22 November 2023  Sesi Pagi [S1iU7QQQ-BU]_000_300.mp3'},
 {'question': 'transcribe the audio into Whisper format',
  'answer': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Naik tarap hospital Miri dan hospital Sibu<|3.76|><|5.16|> ke tarap hospital negeri.<|6.80|><|8.36|> Sama tarap<|9.18|><|9.48|> dengan<|9.82|><|10.58|> hospital umum<|11.50|><|12.34|> Sarawak di Kuching.<|13.40|><|endoftext|>',
  'audio_filename': 'parlimen-24k-chunk_processed/parlimen-24k-LANGSUNG  Persidangan Dewan Negara 14 Oktober 2021   Sesi Petang [p2r

In [72]:
alignment[-1]

{'question': 'transcribe the audio into Whisper format in word timestamp',
 'answer': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> Masjid<|0.38|><|0.44|> dibakar,<|0.88|><|1.04|> kampung<|1.44|><|1.50|> dibakar,<|1.98|><|2.26|> anak-anak<|2.74|><|2.82|> dibunuh,<|3.26|><|3.40|> orang<|3.64|><|3.72|> tua<|3.90|><|4.06|> dibunuh.<|4.42|><|4.98|> Mereka<|5.16|><|5.24|> pilih<|5.44|><|5.48|> tempat<|5.68|><|5.72|> kita.<|5.90|><|6.76|> Jadi,<|6.90|><|7.02|> mereka<|7.28|><|7.40|> ini,<|7.76|><|endoftext|>',
 'audio_filename': 'parlimen-24k-chunk_processed/parlimen-24k-LANGSUNG Persidangan Dewan Rakyat  Mesyuarat Kedua Penggal Ketiga  1 Julai 2024  Sesi Petang [JZLbr5Nbxrs]_001/parlimen-24k-LANGSUNG Persidangan Dewan Rakyat  Mesyuarat Kedua Penggal Ketiga  1 Julai 2024  Sesi Petang [JZLbr5Nbxrs]_001_115.mp3'}

In [74]:
import pandas as pd

pd.DataFrame(alignment).to_parquet('more-stt.parquet')

In [75]:
!huggingface-cli upload mesolitica/Transcription-Instructions \
more-stt.parquet /data/extra_malaysian-00000-of-00001.parquet --repo-type=dataset

Uploading files using Xet Storage..
Uploading...: 100%|██████████████████████████| 144M/144M [00:14<00:00, 9.99MB/s]
https://huggingface.co/datasets/mesolitica/Transcription-Instructions/blob/main//data/extra_malaysian-00000-of-00001.parquet
